# Phase DV Chip - Detecting real life Violence using CV

In [1]:
import os
import cv2
import numpy as np
from imutils import paths

In [2]:
from sklearn.preprocessing import LabelBinarizer

# Importing datapath 

In [3]:
datapath="video dataset"
outputmodel="output"
outputlabelbinarizer="output\videoclassificationbinarizer"

In [4]:
import cv2
import os
import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")

from keras.layers import Input
from keras.models import Model
from keras.layers.core import Dropout,Flatten,Dense

import matplotlib.pyplot as plt

In [5]:
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")

# Predict video for Violence/Non-Violence

In [6]:
import numpy as np
import argparse
import pickle
import cv2
import os
import time 
from keras.models import load_model
from collections import deque

def print_results(video, limit=None):
        #fig=plt.figure(figsize=(16, 30))
        if not os.path.exists('output'):
            os.mkdir('output')

        print("Loading model ...")
        model = load_model('model.h5')
        Q = deque(maxlen=64)
        vs = cv2.VideoCapture(video)
        writer = None
        (W, H) = (None, None)
        count = 0     
        while True:
            # read the next frame from the file
            (grabbed, frame) = vs.read()

            # if the frame was not grabbed, then we have reached the end
            # of the stream
            if not grabbed:
                break

            # if the frame dimensions are empty, grab them
            if W is None or H is None:
                (H, W) = frame.shape[:2]

            # clone the output frame, then convert it from BGR to RGB
            # ordering, resize the frame to a fixed 224x224, and then
            # perform mean subtraction
            
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            output = cv2.resize(frame,(512,360)).copy()
            frame = cv2.resize(frame, (128, 128)).astype("float32")
            frame = frame.reshape(128,128,3)/255
            frame -= mean

            # make predictions on the frame and then update the predictions
            # queue
            preds = model.predict(np.expand_dims(frame, axis=0))[0]
            Q.append(preds)

            # perform prediction averaging over the current history of
            # previous predictions
            results = np.array(Q).mean(axis=0)
            # i = np.argmax(results)
            i = (preds > 0.56)[0]
            label = i


            # draw the activity on the output frame
            # prob = model.predict_proba(np.expand_dims(frame, axis=0))[0] # to show probability of frame
            prob = results[i]*100

            text_color = (0, 255, 0) # default : green

            if prob > 70 : # Violence prob
                text_color = (0, 0, 255) # red

            else:
                label = 'Normal'

            #text = "State : {:8} ({:3.2f}%)".format(label,prob)
            text = "Violence: {}".format(label)
            FONT = cv2.FONT_HERSHEY_SIMPLEX 

            cv2.putText(output, text, (35, 50), FONT,1.25, text_color, 3) 

            # plot graph over background image
            output = cv2.rectangle(output, (35, 80), (35+int(prob)*5,80+20), text_color,-1)

            # check if the video writer is None
            if writer is None:
                # initialize our video writer
                fourcc = cv2.VideoWriter_fourcc(*"MJPG")
                writer = cv2.VideoWriter("output/output.avi", fourcc, 30,(W, H), True)

            # write the output frame to disk
            writer.write(output)

            # show the output image
            cv2.imshow("Output", output)
            key = cv2.waitKey(1) & 0xFF

            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break
        # release the file pointersq
        print("[INFO] cleaning up...")
        writer.release()
        vs.release()

In [7]:
Violence="video dataset/Violence/V_1.mp4"

# Output

In [8]:
 print_results(Violence) 

Loading model ...
[INFO] cleaning up...
